# Design Automation Workitem Signing Guide

## Overview

This notebook guides you through the process of **securing your Design Automation workitems** using digital signatures. When using **3-Legged OAuth tokens** (user context) with Design Automation, you must sign your workitem requests to prevent Cross-Site Scripting (XSS) attacks and ensure the integrity of your workflow.

---

## Why Workitem Signing is Required

Design Automation supports two authentication methods:

| Authentication Type | Token Type | Signing Required |
|---------------------|------------|------------------|
| **2-Legged OAuth (2LO)** | Application context | ❌ No |
| **3-Legged OAuth (3LO)** | User context | ✅ Yes |

When using 3LO tokens, an attacker could potentially inject malicious activity IDs into your workitem requests. The digital signature mechanism prevents this by:

1. **Verifying the caller's identity** - Only the owner of the private key can create valid signatures
2. **Ensuring message integrity** - Any modification to the activity ID invalidates the signature
3. **Binding the workitem to your application** - The signature is tied to your APS client credentials

---

## Complete Signing Workflow

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                      DESIGN AUTOMATION SIGNING WORKFLOW                     │
└─────────────────────────────────────────────────────────────────────────────┘

  ┌─────────────┐         ┌─────────────┐         ┌─────────────┐
  │   STEP 1    │         │   STEP 2    │         │   STEP 3    │
  │ Generate    │────────▶│ Upload      │────────▶│ Sign        │
  │ Key Pair    │         │ Public Key  │         │ Workitems   │
  └─────────────┘         └─────────────┘         └─────────────┘
        │                       │                       │
        ▼                       ▼                       ▼
  Das.WorkItemSigner     PATCH forgeapps/me      Das.WorkItemSigner
  generate mykey.json    {publicKey: ...}        sign mykey.json <activityId>
        │                       │                       │
        ▼                       ▼                       ▼
  ┌─────────────┐         ┌─────────────┐         ┌─────────────┐
  │ mykey.json  │         │ APS Server  │         │ POST        │
  │ (private)   │         │ stores key  │         │ /workitems  │
  │             │         │             │         │ with header │
  │ mypublickey │         │             │         │ x-ads-      │
  │ .json       │         │             │         │ workitem-   │
  │ (public)    │         │             │         │ signature   │
  └─────────────┘         └─────────────┘         └─────────────┘
```

---

## Tools & Prerequisites

### Das.WorkItemSigner Tool

The **Das.WorkItemSigner** is a .NET utility provided by Autodesk to:
- Generate RSA key pairs for signing
- Export public keys in the required format
- Create digital signatures for activity IDs

**Download**: [Latest Release (v20221208)](https://github.com/autodesk-platform-services/aps-designautomation-signer/releases/tag/release-main-20221208-102554)

**Repository**: [autodesk-platform-services/aps-designautomation-signer](https://github.com/autodesk-platform-services/aps-designautomation-signer)

### Commands Overview

| Command | Description |
|---------|-------------|
| `Das.WorkItemSigner.exe generate <keyfile>` | Generate a new RSA key pair |
| `Das.WorkItemSigner.exe export <keyfile> <pubkeyfile>` | Export public key for upload |
| `Das.WorkItemSigner.exe sign <keyfile> <activityId>` | Sign an activity ID |

---

## API Reference

### PATCH forgeapps/{id}

The `PATCH forgeapps/me` endpoint is used to:
- Set your application nickname
- Upload your public key for workitem signature verification

**Endpoint**: `https://developer.api.autodesk.com/da/{region}/v3/forgeapps/me`

**Request Body** (for public key upload):
```json
{
  "publicKey": {
    "Exponent": "AQAB",
    "Modulus": "..."
  }
}
```

**Documentation**: [PATCH forgeapps-id](https://aps.autodesk.com/en/docs/design-automation/v3/reference/http/forgeapps-id-PATCH/)

---

## Security Considerations

| ⚠️ Security Best Practices |
|---------------------------|
| **Never commit `mykey.json`** to version control - it contains your private key |
| **Store private keys securely** - use environment variables or secure vaults |
| **Rotate keys periodically** - upload a new public key to replace the old one |
| **Use separate keys** for development and production environments |

---

## Workflow Steps in This Notebook

1. **Step 1**: Generate key pair using `Das.WorkItemSigner.exe`
2. **Step 2**: Upload public key to APS via `PATCH forgeapps/me`
3. **Step 3**: Sign workitems when submitting with 3LO tokens

---

## References

- [3-Legged OAuth Token Usage (Design Automation)](https://aps.autodesk.com/en/docs/design-automation/v3/developers_guide/3-legged-oauth-token-usage/)
- [PATCH forgeapps/{id} API Reference](https://aps.autodesk.com/en/docs/design-automation/v3/reference/http/forgeapps-id-PATCH/)
- [Design Automation Signer Tool (GitHub)](https://github.com/autodesk-platform-services/aps-designautomation-signer)
- [Signer Tool Releases](https://github.com/autodesk-platform-services/aps-designautomation-signer/releases)

## Imports

## Step 1: Generate Keys

The first step is to generate an RSA key pair using the **Das.WorkItemSigner** tool. This tool uses industry-standard RSA cryptography to create:

- A **private key** (`mykey.json`) - Used to sign workitems (keep this secret!)
- A **public key** (`mypublickey.json`) - Uploaded to APS for signature verification

### Generate the Keys

Run these commands in PowerShell where you have the `Das.WorkItemSigner.exe`:

```powershell
# Generate private key (contains both private and public key data)
.\\Das.WorkItemSigner.exe generate mykey.json

# Export public key to a separate file for upload
.\\Das.WorkItemSigner.exe export mykey.json mypublickey.json
```

> 💡 **Tip**: Download `Das.WorkItemSigner.exe` from the [releases page](https://github.com/autodesk-platform-services/aps-designautomation-signer/releases/tag/release-main-20221208-102554)

### Key File Structures

**Private Key (`mykey.json`)** - Keep this secret! Contains RSA key components:

```json
{
  "D": "...",           // Private exponent (SECRET)
  "DP": "...",          // First factor CRT exponent
  "DQ": "...",          // Second factor CRT exponent  
  "Exponent": "AQAB",   // Public exponent (65537 in base64)
  "InverseQ": "...",    // CRT coefficient
  "Modulus": "...",     // RSA modulus (shared with public key)
  "P": "...",           // First prime factor
  "Q": "..."            // Second prime factor
}
```

**Public Key (`mypublickey.json`)** - This gets uploaded to APS:

```json
{
  "Exponent": "AQAB",   // Public exponent (standard RSA value)
  "Modulus": "..."      // RSA modulus (2048-bit key)
}
```

> ⚠️ **Important**: Never share or commit `mykey.json` - it contains your private key!

## Step 2: Upload Public Key to APS

Now we'll upload the public key to your APS application. This registers the key so APS can verify your signed workitems.

### Imports

In [ ]:
import os
import json
from pathlib import Path
import requests
from dotenv import load_dotenv

APS = "https://developer.api.autodesk.com"

## Helper Functions

These utility functions handle the APS API interactions:

| Function | Purpose |
|----------|---------|
| `get_2lo_token()` | Obtains a 2-Legged OAuth token with `code:all` scope |
| `get_forgeapps_me()` | Retrieves current app nickname or client ID |
| `try_set_nickname()` | Attempts to set a custom nickname for the app |
| `upload_public_key()` | Uploads the public key via PATCH forgeapps/me |

In [ ]:
def get_2lo_token(client_id: str, client_secret: str) -> str:
    r = requests.post(
        f"{APS}/authentication/v2/token",
        data={
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": "code:all",
        },
        timeout=30,
    )
    r.raise_for_status()
    return r.json()["access_token"]

In [ ]:
def get_forgeapps_me(region: str, token: str) -> str:
    url = f"{APS}/da/{region}/v3/forgeapps/me"
    r = requests.get(url, headers={"Authorization": f"Bearer {token}"}, timeout=30)
    r.raise_for_status()
    try:
        return r.json()  # string (nickname or client_id)
    except ValueError:
        return r.text.strip('" \n\r')

In [ ]:
def try_set_nickname(region: str, token: str, nickname: str) -> None:
    url = f"{APS}/da/{region}/v3/forgeapps/me"
    r = requests.patch(
        url,
        headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
        json={"nickname": nickname},
        timeout=30,
    )
    if r.status_code == 409:
        # App already has DA resources. Nickname cannot change.
        print("Nickname change skipped: app already has resources. Continue with existing nickname.")
        return
    r.raise_for_status()

In [ ]:
def upload_public_key(region: str, token: str, public_key_json: dict) -> None:
    url = f"{APS}/da/{region}/v3/forgeapps/me"
    r = requests.patch(
        url,
        headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
        json={"publicKey": public_key_json},
        timeout=30,
    )
    r.raise_for_status()

## Configuration

We use `python-dotenv` to load credentials from a `.env` file. This keeps your secrets out of the code!

### Create your `.env` file

Create a file named `.env` in the same directory as this notebook (or project root):

```env
# APS Application Credentials (from https://aps.autodesk.com/myapps)
CLIENT_ID=your_client_id_here
CLIENT_SECRET=your_client_secret_here

# Design Automation region (optional, defaults to us-east)
REGION=us-east
```

> ⚠️ **Important**: Add `.env` to your `.gitignore` to prevent committing secrets!

### Configuration Variables

| Variable | Description | Required |
|----------|-------------|----------|
| `CLIENT_ID` | Your APS application client ID | ✅ Yes |
| `CLIENT_SECRET` | Your APS application client secret | ✅ Yes |
| `REGION` | DA region: `us-east` or `us-west` | No (default: `us-east`) |

### About the Nickname

The `desired_nickname` is a **user-defined, globally unique** identifier for your Design Automation app:

- **Must be unique** across ALL APS applications worldwide
- **Cannot be changed** once you have created any DA resources (activities, app bundles)
- **Used in activity IDs** - format: `<nickname>.<ActivityName>+<alias>`
- If not set, your `CLIENT_ID` will be used as the identifier

> 💡 **Tip**: Choose a meaningful, unique nickname like `companyname_projectname` before creating any activities!

In [ ]:
load_dotenv()

# Load credentials from .env file
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
print(client_secret)
print(client_id)
region = os.getenv("REGION", "us-east")

# ⚠️ IMPORTANT: Set your unique nickname here!
# - Must be globally unique across all APS applications
# - Cannot be changed after creating DA resources (activities, bundles)
# - Will be used in activity IDs: <nickname>.<ActivityName>+<alias>
desired_nickname = "viktorautomationtest"  # <-- Change this to your unique nickname!

# Path to the public key file generated by Das.WorkItemSigner (assumes notebook cwd)
base_dir = Path.cwd()
key_path = base_dir / "mypublickey.json"

# Validation
if not client_id or not client_secret:
    raise ValueError("Missing CLIENT_ID or CLIENT_SECRET in .env file")
if not key_path.exists():
    raise FileNotFoundError(f"Public key file not found: {key_path.resolve()}")

public_key_json = json.loads(key_path.read_text(encoding="utf-8"))
if not isinstance(public_key_json, dict) or not public_key_json:
    raise ValueError("Public key JSON must be a non-empty object exported by the signer.")

print("== Configure DA app ==")
print(f"Base dir        : {base_dir}")
print(f"Region          : {region}")
print(f"Client ID       : {client_id}")
print(f"Desired Nickname: {desired_nickname}")

## Get Token

In [ ]:
token = get_2lo_token(client_id, client_secret)
print("Token obtained successfully")

## Get Current App Info

In [ ]:
current_name = get_forgeapps_me(region, token)
print(f"forgeapps/me: {current_name!r}")

## Set Nickname (Optional)

In [ ]:
if desired_nickname and desired_nickname != current_name:
    print(f"Attempting to set nickname to: {desired_nickname}")
    try_set_nickname(region, token, desired_nickname)
    current_name = get_forgeapps_me(region, token)
    print(f"Nickname now: {current_name!r}")
else:
    print(f"Nickname already set to: {current_name!r}")

## Upload Public Key

In [ ]:
print("Uploading public key...")
upload_public_key(region, token, public_key_json)
print("Success: public key is registered for this app and region.")

## Step 3: Sign Workitems

Now that your public key is registered with APS, you can sign workitems using your private key. The signature is **specific to the activity ID** you want to execute.

### How Signing Works

When Design Automation receives a workitem with a 3LO token, it:
1. Extracts the `activityId` from the request body
2. Reads the `x-ads-workitem-signature` header
3. Verifies the signature using your registered public key
4. Only executes the workitem if the signature is valid

This ensures that only **you** (the owner of the private key) can execute activities under your application.

### Generate Signature

Use the signer tool to create a signature for your activity. The format is:

```
<nickname_or_clientId>.<ActivityName>+<alias>
```

```powershell
.\\Das.WorkItemSigner.exe sign mykey.json <nickname>.BuildStructureActivity+prod
```

**Example**:
```powershell
.\\Das.WorkItemSigner.exe sign mykey.json viktortest.BuildStructureActivity+prod
```

**Output** (example signature):
```
NEBfeSlrr56uCGqQ+3yuKi+CU/VKEsefequ4fBz017MCnI0cydrICD79DgpXGmxVMIuo8ajfCMxzK/SpHA6qnYa5w0S...
```

> 💡 **Note**: The signature is deterministic - the same activity ID with the same private key always produces the same signature. You can pre-generate signatures for all your activities.

### Using the Signature

When posting workitems, include the signature in the `x-ads-workitem-signature` header:

In [ ]:
# Example: Using the signature in workitem requests
ACTIVITY_ID = f"{current_name}.BuildStructureActivity+prod"

# Replace with the output from: Das.WorkItemSigner.exe sign mykey.json <activity_id>
ACTIVITY_SIGNATURE = "YOUR_SIGNATURE_HERE"

print("When posting /workitems, use these headers:")
print(f"  Authorization: Bearer <your_3LO_token>")
print(f"  x-ads-workitem-signature: {ACTIVITY_SIGNATURE}")
print(f"\nActivity ID: {ACTIVITY_ID}")

## Summary

You have successfully completed the Design Automation signing workflow:

| Step | Action | Status |
|------|--------|--------|
| 1 | Generated key pair (`mykey.json` + `mypublickey.json`) | ✅ |
| 2 | Uploaded public key to APS via `PATCH forgeapps/me` | ✅ |
| 3 | Learned how to sign workitems with `Das.WorkItemSigner.exe sign` | ✅ |

---

### Key Takeaways

- **Signing is required** when using 3-Legged OAuth tokens with Design Automation
- **Keep `mykey.json` secret** - it's your private key for signing
- **Signatures are activity-specific** - format is `nickname.ActivityName+alias`
- **Public key can be updated** - simply PATCH a new one to replace the old

### Troubleshooting

| Issue | Solution |
|-------|----------|
| Signature verification failed | Ensure you're signing the exact activity ID format |
| 401 Unauthorized | Check that your token has `code:all` scope |
| 409 Conflict on nickname | App already has DA resources, use existing nickname |

---

### Quick Reference Links

| Resource | Link |
|----------|------|
| 3LO Token Usage Guide | [Documentation](https://aps.autodesk.com/en/docs/design-automation/v3/developers_guide/3-legged-oauth-token-usage/) |
| PATCH forgeapps API | [API Reference](https://aps.autodesk.com/en/docs/design-automation/v3/reference/http/forgeapps-id-PATCH/) |
| Signer Tool | [GitHub Repository](https://github.com/autodesk-platform-services/aps-designautomation-signer) |
| Latest Release | [Download](https://github.com/autodesk-platform-services/aps-designautomation-signer/releases/tag/release-main-20221208-102554) |